In [297]:
### Importing relevent lib ###
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import bs4
from bs4 import BeautifulSoup 
import requests
import scipy as sc
import numpy as np
import re


In [298]:
### get all genre links to scrapt information of each genre Anime in myAnimeList ###
def get_all_genre_links(site_url,genre_dic):
    genre_url = []
    for i in genre_dict:
        genre_url.append(site_url + genre_dict[i]+ '/'+i)
    return genre_url

### All Genres in Site ###
genre_dict = {
    'Action': '1',
    'Adventure':'2',
    'Cars':'3',
    'Comedy':'4',
    'Dementia':'5',
    'Demons':'6',
    'Drama':'8',
    'Ecchi':'9',
    'Fantasy':'10',
    'Game':'11',
    'Historical':'13',
    'Horror':'14',
    'Kids':'15',
    'Harem':'35',
    'Josei':'43'
}

In [299]:
### get all top anime links ###
### Sample works on 10,000 diffrent anime ###
def get_all_top_links(site_url):
    all_top_urls = []
    for i in range(0,10000,50):
        all_top_urls.append(site_url+'?limit='+str(i))
    return all_top_urls

In [300]:
### Load a soup Object ###
def load_soup_obj(html):
    html_get = requests.get(html).text
    soup = BeautifulSoup(html_get,'lxml')
    return sou

In [301]:
### Site's url's ###
mal_genre_front_page = 'https://myanimelist.net/anime.php'
top_anime_url = 'https://myanimelist.net/topanime.php'
### All genre Links list ###
mal_genre_links = get_all_genre_links(mal_genre_front_page,genre_dict)
### All Top 0-19,000 Anime's in mal URL list ###
top_anime_list = get_all_top_links(top_anime_url)


In [302]:
### Engline The Episodes ###
def parse_episodes(listt):
    result = []
    for i in listt[:3]:
        r = i.strip()
#         print(r)
        result.append(r)
    return result

In [352]:
def delete_ep_string(list_ep):
    
    
    res = []

    for i,_ in enumerate(list_ep):
        search_TV = re.search('TV',list_ep[i])
        search_OVA = re.search('OVA',list_ep[i])
        search_Movie = re.search('Movie',list_ep[i])
        if search_TV:
            a = list_ep[i].split('TV')[1]
            res.append(a)
        elif search_OVA:
            a = list_ep[i].split('OVA')[1]
            res.append(a)
        elif search_Movie:
            a = list_ep[i].split('Movie')[1]
            res.append(a)
        else:
            res.append(np.nan)
            continue
    return res


In [353]:
def delete_members_string(list_mem):
    
    res = []
    
    for i,_ in enumerate(list_mem):
        search = re.search('members',list_mem[i])
        if search:
            a = list_mem[i].split('members')[0]
            res.append(a)    
    return res

In [396]:
def split_dates(list_dates):
    result_s = []
    result_f = []
    
    
    for i,_ in enumerate(list_dates):
        res_s = list_dates[i].split('-')[0]
        res_f = list_dates[i].split('-')[1]
        result_s.append(res_s)
        result_f.append(res_f)
    
    result = {'Start':result_s,'Finish':result_f}
    return result
    

In [430]:
def split_type(list_ep):
    
    
    res = []

    for i,_ in enumerate(list_ep):
        search_TV = re.search('TV',list_ep[i])
        search_OVA = re.search('OVA',list_ep[i])
        search_Movie = re.search('Movie',list_ep[i])
        if search_TV:
#             a = list_ep[i].split('TV')[1]
            res.append(search_TV.group(0))
        elif search_OVA:
#             a = list_ep[i].split('OVA')[1]
            res.append(search_OVA.group(0))
        elif search_Movie:
#             a = list_ep[i].split('Movie')[1]
            res.append(search_Movie.group(0))
        else:
            res.append(np.nan)
            continue
    return res

        
    

In [ ]:
top_anime = []
top_anime_Rank = []
top_anime_Title = []
top_anime_Rating = []
top_anime_URL = []
top_anime_Image_URL = []
top_anime_Episodes = []
top_anime_Dates = []
top_anime_m_count = []
try_list = []
counter = 0

### inspierd by https://blog.jovian.ai/scraping-data-using-beautiful-soup-and-python-4170e7ec63fd ###
### inspierd by ###

for link in top_anime_list:
    
    ### Load the soup object trough load_soup_obj() ###
    html_get = requests.get(link)
    t_html_get = html_get.text
    counter+=1
    if html_get.status_code != 200:
            print('Status code:', html_get.status_code)
    
    print(counter)
    time.sleep(2)
    soup = BeautifulSoup(t_html_get,'lxml')
    soup.title.text.strip()
    
    ### find all Rank, Title, Rating, Date of Release, Episodes, Image_Url ###
    headers_mal = soup.find('tr', class_ = 'table-header')
    ### extract Rank, Title, Rating, Date of Release, Episodes, Image_Url from the given HTML ###
    row_content = soup.find_all('tr', {'class' : "ranking-list"})
    #print(row_content)

    for row in row_content:
        episode = parse_episodes(row.find('div', class_ = "information di-ib mt4").text.strip().split('\n'))

        
        
        top_anime_URL.append(row.find('td', class_= "title al va-t word-break").find('a')['href'])
        top_anime_Rank.append(row.find('td', class_ = "rank ac").find('span').text)
        top_anime_Title.append(row.find('div', class_="di-ib clearfix").find('a').text)
        top_anime_Rating.append(row.find('td', class_="score ac fs14").find('span').text)
        top_anime_Image_URL.append(row.find('td', class_ ='title al va-t word-break').find('img')['data-src'])
        top_anime_Episodes.append(episode[0])
        top_anime_Dates.append(episode[1]) 
        top_anime_m_count.append(episode[2])
        

        

top_anime_df = pd.DataFrame({'Rank':top_anime_Rank,'Title':top_anime_Title,
                                 'Rating':top_anime_Rating,'Image_URL':top_anime_Image_URL
                                 ,'Episodes':top_anime_Episodes,'Dates':top_anime_Dates,
                                'URL':top_anime_URL,'Members_Votes':top_anime_m_count},)
        



In [434]:

### First Handle of Data ###
copy = top_anime_df.copy()
list_dates = copy.Dates.to_list()
top_anime_df['Episodes'][0]
list_types = split_type(top_anime_df['Episodes'])


### Handle Data ###
copy['Type'] = list_types
copy['Members_Votes'] = delete_members_string(copy['Members_Votes'])
copy['Episodes'] = delete_ep_string(copy['Episodes'])
copy.set_index('Rank',inplace=True)
dic_dates = split_dates(list_dates)
copy['Start Date'] = dic_dates['Start']
copy['Finish Date'] = dic_dates['Finish']
copy.drop('Image_URL',axis=1, inplace=True)
copy.drop('Dates',axis=1, inplace=True)


copy





In [435]:

read_copy = pd.read_csv('Anime_DataFrame_VA_1.csv',index_col='Rank')
read_copy



,Title,Rating,Episodes,URL,Members_Votes,Type,Start Date,Finish Date
Rank,,,,,,,,
1,Fullmetal Alchemist: Brotherhood,9.15,(64 eps),https://myanimelist.net/anime/5114/Fullmetal_A...,"2,720,933",TV,Apr 2009,Jul 2010
2,Gintama°,9.09,(51 eps),https://myanimelist.net/anime/28977/Gintama°,"495,687",TV,Apr 2015,Mar 2016
3,Shingeki no Kyojin Season 3 Part 2,9.09,(10 eps),https://myanimelist.net/anime/38524/Shingeki_n...,"1,650,249",TV,Apr 2019,Jul 2019
4,Steins;Gate,9.09,(24 eps),https://myanimelist.net/anime/9253/Steins_Gate,"2,122,353",TV,Apr 2011,Sep 2011
5,Fruits Basket: The Final,9.06,(13 eps),https://myanimelist.net/anime/42938/Fruits_Bas...,"293,483",TV,Apr 2021,Jun 2021
...,...,...,...,...,...,...,...,...
9996,Idol Project,5.63,(4 eps),https://myanimelist.net/anime/2072/Idol_Project,"1,883",OVA,Sep 1995,Oct 1997
9997,Kekkaishi,5.63,NaN,https://myanimelist.net/anime/40347/Kekkaishi,471,NaN,2004,2004
9998,Kodomo no Keijijougaku,5.63,NaN,https://myanimelist.net/anime/8333/Kodomo_no_K...,"1,022",NaN,Oct 2007,Oct 2007


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           100 non-null    int64  
 1   Title          100 non-null    object 
 2   Rating         100 non-null    float64
 3   Image_URL      100 non-null    object 
 4   Episodes       100 non-null    object 
 5   Dates          100 non-null    object 
 6   URL            100 non-null    object 
 7   Members_Votes  100 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 6.4+ KB


In [186]:
# s = str('Genres: Action, Adventure, Comedy, Drama, Fantasy')
# print(s)

# s2 = s.replace('Genres:','')
# print(s2)
# # str.replace('')

Genres: Action, Adventure, Comedy, Drama, Fantasy
 Action, Adventure, Comedy, Drama, Fantasy


In [9]:
# display(top_anime_df)
# # top_anime_df.to_csv('top_anime_df_v1.csv',)


# # display(top_anime_df.describe())

# top_anime_df_copy = top_anime_df.copy()


# list_top_anime_urls = top_anime_df_copy['URL'].to_list()
# counter = 0

# for l in list_top_anime_urls:

#     ### Load the soup object trough load_soup_obj() ###
#     html_get = requests.get(l)
#     t_html_get = html_get.text
    
    
#     counter+=1
#     print(html_get.status_code)
#     print(counter)
#     time.sleep(2)
    
#     soup = BeautifulSoup(t_html_get,'lxml')
#     soup.title.text.strip()

#     ### find all Rank, Title, Rating, Date of Release, Episodes, Image_Url ###
#     headers_mal2 = soup.find_all('td', class_ = 'borderClass')
#     ### extract Rank, Title, Rating, Date of Release, Episodes, Image_Url from the given HTML ###
#     row_content2 = soup.find_all('div', {"class":'spaceit_pad'})


#     information = []

#     for row in row_content2:
#         if row.find('a') != None:
#             information.append(row.find('a').text)

# print(information[:])


0        OVA (3 eps)
1        OVA (3 eps)
2        OVA (3 eps)
3        OVA (3 eps)
4        OVA (3 eps)
            ...     
18995    OVA (3 eps)
18996    OVA (3 eps)
18997    OVA (3 eps)
18998    OVA (3 eps)
18999    OVA (3 eps)
Name: Episodes, Length: 19000, dtype: object

In [66]:
from mal import AnimeSearch

mal_id_list = []
counter2 = 0 
for i in top_anime_df_copy['Title']:
    search = AnimeSearch(i)
    try:
        mal_id_list.append(search.results[0].mal_id)
    except:
        print(f"Faild at {i} !")
        continue
    
    
    


Fullmetal Alchemist: Brotherhood.5114
Gintama°.28977
Shingeki no Kyojin Season 3 Part 2.38524
Steins;Gate.9253
Fruits Basket: The Final.42938
Gintama'.7472
Hunter x Hunter (2011).11061
Ginga Eiyuu Densetsu.820
Gintama': Enchousen.15417
Gintama: The Final.15335
Gintama..7472
Violet Evergarden Movie.37987
3-gatsu no Lion 2nd Season.35180
Koe no Katachi.28851
Clannad: After Story.4181
Gintama.7472
Gintama Movie 2: Kanketsu-hen - Yorozuya yo Eien Nare.15335
Code Geass: Hangyaku no Lelouch R2.5163
Owarimonogatari 2nd Season.35247
Shingeki no Kyojin: The Final Season.40028
Kimi no Na wa..32281
Gintama.: Shirogane no Tamashii-hen - Kouhan-sen.37491
Ousama Ranking.40834
Shiguang Dailiren.44074
Gintama.: Shirogane no Tamashii-hen.37491
Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou.32935
Kizumonogatari III: Reiketsu-hen.31758
Mob Psycho 100 II.50172
Monster.14227
Mushoku Tensei: Isekai Ittara Honki Dasu Part 2.45576
Sen to Chihiro no Kamikakushi.199
Monogatari Series: Second Season.

ValueError: No results found

In [ ]:
from mal import Anime

genre_list = []
scored_by_list = []
theme_list = []
prducers_list = []


for i in mal_id_list:
    anime = Anime(i)
    genre_list.append(anime.genres)
    scored_by_list.append(anime.scored_by)
    theme_list.append(anime.themes)
    prducers_list.append(anime.prducers)


    
    

In [161]:
list_top_anime_urls = top_anime_df_copy['URL'].to_list()
counter3 = 0
for anime in list_top_anime_urls:
    ### Load the soup object trough load_soup_obj() ###
    html_get = requests.get(anime)
    t_html_get = html_get.text
    if (html_get.status_code != 200):
        print('Status code:', response.status_code)
        raise Exception(f"Failed to fetch web page{anime}")
        
    time.sleep(5)
    counter3+=1
    print(counter3)
    soup = BeautifulSoup(t_html_get,'lxml')
    soup.title.text.strip()

    ### find all Rank, Title, Rating, Date of Release, Episodes, Image_Url ###
    headers_mal2 = soup.find_all('td', class_ = 'borderClass')
    ### extract Rank, Title, Rating, Date of Release, Episodes, Image_Url from the given HTML ###
    row_content2 = soup.find_all('div', {"class":'spaceit_pad'})


    information = []

    for row in row_content2:
        if row.find('a') != None:
            information.append(row.find('a').text)
            
    if counter3 == 500 :
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [165]:
print(len(information))

7


In [153]:
# def get_anime_df(topic):
#     site_url = 'https://myanimelist.net/anime/genre/'
    
#     def get_topic_page(genre):
#         final_url = site_url + genre_dict[genre]+ '/' + genre
#         response = requests.get(final_url)
#         if response.status_code != 200:
#             print('Status code:', response.status_code)
            
#             raise Exception('Failed to fetch web page ' + final_url)
#         return BeautifulSoup(response.text)  
    
#     doc = get_topic_page(topic)
    
#     titles = doc.find_all('h2', class_ = 'h2_anime_title')
#     producers = doc.find_all('span', class_ ='producer')
#     episodes = doc.find_all('div', class_='eps')
#     images = doc.find_all('div', class_='image')
#     ratings = doc.find_all('div', class_= 'scormem')
#     members = doc.find_all('div', class_= 'scormem')
    
#     def create_lists():
#         list_titles = []; list_producer = []; list_episodes = []; list_images = []; list_ratings = []; list_members =[]

#         for title in titles:
#             name = title.find('a').text
#             list_titles.append(name)
#         for producer in producers:
#             tag = producer.find('a')['title']
#             list_producer.append(tag)
#         for episode in episodes:
#             e = episode.find('span').text
#             list_episodes.append(e)
#         for img in images:
#             i = img.find('img')['data-src']
#             list_images.append(i)
#         for rating in ratings:
#             r = rating.find('span', class_='score').text.strip()
#             list_ratings.append(r)
#         for member in members:
#             m = member.find('span', class_='member').text.strip()
#             list_members.append(m)
        
#         return list_titles, list_producer, list_episodes, list_images , list_ratings , list_members

#     if len(titles) == len(producers) == len(episodes) == len(images) == len(ratings) == len(members):
#         list_titles, list_producer, list_episodes, list_images , list_ratings , list_members = create_lists()
        
#         column_names = ['Title', 'Producer', 'Episodes', 'Rating', 'Members','Url']
#         df = pd.DataFrame(list(zip(list_titles, list_producer, list_episodes, list_ratings, list_members, list_images)), columns = column_names)
        
        
#     else: print("Length of lists are not equal, dataframe cannot be created")
     
    
#     return df
    

In [26]:
# ### Selenium Go Trough Pages ###

# ### Open Certain Page in Selenium ###
# PATH = "C:/Users/Kobi/Downloads/chromedriver.exe"
# driver = webdriver.Chrome(PATH)
# driver.get('https://myanimelist.net/topanime.php')

# ### Lists ###
# top_anime = []
# top_anime_Rank = []
# top_anime_Title = []
# top_anime_Rating = []
# top_anime_URL = []
# top_anime_Image_URL = []
# top_anime_Episodes = []
# top_anime_Dates = []


# html = driver.page_source
# soup = BeautifulSoup(html)

# soup.title.text.strip()


# ### find all Rank, Title, Rating, Date of Release, Episodes, Image_Url ###
# headers_mal = soup.find('tr', class_ = 'table-header')
# ### extract Rank, Title, Rating, Date of Release, Episodes, Image_Url from the given HTML ###
# row_content = soup.find_all('tr', {'class' : "ranking-list"})

# ### Fill Lists ###
# top_anime_dic = bs_to_dic(row_content=row_content)

# time.sleep(2)



# try:
#     main = WebDriverWait(driver, 20).until(EC.presence_of_element_located(
#         (By.CLASS_NAME,'pb12')))
#     link = driver.find_element_by_link_text("Next 50")
#     link.click()
    
# finally:
#     driver.quit()


# # link = driver.find_element_by_link_text('Next 50')
# # link.click()



In [27]:
### TEST ###
# for i,_ in enumerate(top_anime_list):
#     html = load_top_anime_html(_)
#     res = html_to_lists(html)
#     if (i>=1):
#         df_new = html_to_lists(html[i])
#         df_merged = pd.merge(res,df_new)
    

#res = load_top_anime(top_anime_list[1])
#print(top_anime_list[1])
#merged_df = pd.merge(top_anime_df_2,res,on=['Rank', 'Title', 'Rating', 'Image_URL', 'Episodes', 'Dates', 'URL'])


In [28]:
# ### Load the soup object trough load_soup_obj() ###
# soup = load_soup_obj('https://myanimelist.net/topanime.php?limit=100')
# soup.title.text.strip()
# ### find all Rank, Title, Rating, Date of Release, Episodes, Image_Url ###
# headers_mal = soup.find('tr', class_ = 'table-header')
# ### extract Rank, Title, Rating, Date of Release, Episodes, Image_Url from the given HTML ###
# row_content = soup.find_all('tr', {'class' : "ranking-list"})